# Knowledge Data Discovery and Neural Networks : Final Project

In this notebook we will do some basic data analysis in python 

## Clarifications
1. The same score for the project will be given to each member of the team.  
2. When you are asked to provide a verbal solution - write it concisely and short (a couple of sentences).
3. Data decription can be found [here](https://archive.ics.uci.edu/ml/datasets/adult)

# 1. Imports

In [1]:
# add more packages in this section
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 2. Exploratory data analysis

In [2]:
# data = pd.read_csv("data/adult.data", header = None)
data = pd.read_csv("https://raw.githubusercontent.com/Galzi1/DataDiscoveryNNProj/master/example_submission.csv", header = None)
cols = ['age', 'workclass', 'fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain', 'capital_loss'
    ,'hours_per_week','native_country','y']
data.columns = cols

FileNotFoundError: ignored

## 2.1 Row and column count
* How many rows and columns/features we have in the dataset? 

In [ ]:
### YOUR CODE HERE



### END YOUR CODE

## 2.2 Missing values proporiton
* For each feature - what is the percantage of missing values in it?
* Describe one method to fill missing data (data imputation)

If there will be many missing values then we will need to take it in consideration later 

In [ ]:
### YOUR CODE HERE



### END YOUR CODE

#### YOUR VERBAL SOLUTION HERE



#### END YOUR VERBAL SOLUTION HERE

## 2.3 Categorical features

* Which of the features are categorical type? for each feature like that, how many unique values it has?

In [ ]:
### YOUR CODE HERE



### END YOUR CODE

## 2.4 Feature distribution

* Plot the distribution of each feature
* Is our data set balanced (do we have the same amount of <=50K and >50K in the label?)

In [ ]:
### YOUR CODE HERE



### END YOUR CODE

## 2.5 Correlation plot

* Plot a correlation plot between the numeric features

In [ ]:
### YOUR CODE HERE



### END YOUR CODE

## 2.6 Important features

* Try to understand which features will be important for the prediction and back it up with a plot or two that shows it.

It is an open question and there are many ways to approach it. 

In [ ]:
### YOUR CODE HERE



### END YOUR CODE

#### YOUR VERBAL SOLUTION HERE



#### END YOUR VERBAL SOLUTION HERE